In [1]:
%matplotlib inline

Testing some notes here.

In [2]:
path = "data/dogscats2/"

In [4]:
from __future__ import division,print_function
import os, json, os.path
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots

Using Theano backend.


ImportError: cannot import name activity_l2

Below here is some new code.
They recommend batch_size=64 or higher, but I'm on t2 right now, so I'll do 16 (to try things out).

In [41]:
batch_size=16
from vgg16 import Vgg16
vgg = Vgg16()

batches = vgg.get_batches(path+'train', batch_size=batch_size)

Found 23000 images belonging to 2 classes.


In [42]:
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size * 2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 2000 images belonging to 2 classes.
Epoch 1/1
  144/23000 [..............................] - ETA: 5994s - loss: 0.7861 - acc: 0.7569

KeyboardInterrupt: 

In [27]:
# CMG HERE--working through the predict.
def pred_batch(imgs):
    preds, indexes, classes = vgg.predict(imgs)

    #print('Shape: {}'.format(preds.shape))
    #print('First 5 classes: {}'.format(classes[:5]))
    #print('First 5 probabilities: {}'.format(preds[:5]))
    #print('Len(preds) {}, Len(indexes) {}, Len(classes) {}'.format(len(preds), len(indexes), len(classes)))
    #print('preds {}, indexes {}, classes {}'.format(preds, indexes, classes))

    #print('\nPredictions prob/class: ')
    dog_preds = []
    for i in range(len(indexes)):
        #print ('  {:.4f}/{}'.format(preds[i], classes[i]))
        if classes[i] == 'cats':
            dog_preds.append(1 - preds[i])
        else:
            dog_preds.append(preds[i])
    return dog_preds


In [28]:
# Need to read in images somehow.
# Get number of files in directory

batch_size=16
test_batches=vgg.get_batches(path+'test', shuffle=False, batch_size=batch_size)
num_files = len(test_batches.filenames)
n = 0
files_done = 0
dog_preds = []
for images,labels in test_batches:
    start = n * batch_size
    if (n + 1) * batch_size > num_files:
        this_batch = num_files - (n * batch_size)
    else:
        this_batch = batch_size
    #print('START {}: SIZE {}'.format(start, this_batch))
    #print(test_batches.filenames[start : start + this_batch])
    dog_preds.extend(pred_batch(images))
    n+=1
    files_done += this_batch
    if files_done >= num_files:
        break

Found 5 images belonging to 1 classes.


In [29]:
ids = list(map(lambda f: filter(str.isdigit, f), test_batches.filenames))
#print(ids)
#print(dog_preds)
#print(test_batches.filenames)

In [32]:
print('Output results')
f = open(path + 'results.csv', 'w')
f.write('id,label\n')
for i in range(len(ids)):
    f.write('{},{}\n'.format(ids[i],dog_preds[i]))
print("Wrote out {} results".format(len(ids)))
f.close()

Output results
Wrote out 5 results
